In [1]:
import pandas as pd
import re

/Users/adrienawong/.pyenv/versions/3.7.0/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/adrienawong/.pyenv/versions/3.7.0/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
file = 'clickstream-enwiki-2020-01.tsv'
folder = '../Datasets/'
path = folder+file

In [3]:
df = pd.read_csv(path, delimiter='\t', 
                       encoding='utf-8', names=['referer', 'resource', 'path', 'count'])

In [4]:
df

,referer,resource,path,count
0,Eddie_Albert,The_Dude_Goes_West,link,17
1,other-empty,The_Dude_Goes_West,external,112
2,Gale_Storm,The_Dude_Goes_West,link,15
3,other-external,Gianluca_Scamacca,external,18
4,other-internal,Gianluca_Scamacca,external,111
...,...,...,...,...
33033625,Government_of_the_Czech_Republic,Alena_Schillerová,link,20
33033626,Himmat_(app),Nirbhaya_(app),link,12
33033627,other-search,Nirbhaya_(app),external,195
33033628,other-empty,Nirbhaya_(app),external,423


In [5]:
# get all external link click count for resource
df_external_count = df.groupby(['resource', 'path'])['count'].sum()
df_external_count = df_external_count.reset_index()
df_external_count = df_external_count.loc[df_external_count['path'] == 'external']
df_external_count['referer'] = 'other-external'

In [6]:
# get all internal link click count for resource
df_internal = df.loc[df['path'] == 'link']
df_internal = df_internal.dropna()

In [7]:
# combine them together
df_combined = pd.concat([df_internal, df_external_count], axis=0)
df_combined = df_combined.sort_values(by=['resource', 'path', 'count']).reset_index(drop=True)

/Users/adrienawong/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [8]:
reg = re.compile(r'[^a-zA-Z0-9\-\_.]')

In [9]:
def preprocess(doc):
    rs = ''
    if(reg.search(doc) == None): 
        rs = doc
    return rs

In [10]:
# get english alphabets, numbers, -, and _ only
df_combined['resource'] = df_combined['resource'].apply(preprocess)
df_result = df_combined[df_combined['resource'].map(len) > 0]

In [11]:
# get date from file name
date = file[-11:-4].replace('-', '')
df_result['date'] = date

/Users/adrienawong/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# reorder and rename the columns
df_result = df_result[['date', 'resource', 'referer', 'count']].rename(
    columns={'resource': 'title', 'referer': 'from'})
df_result = df_result.reset_index(drop=True)

In [13]:
df_result

,date,title,from,count
0,202001,--,other-external,2016
1,202001,--,Hyphen-minus,68
2,202001,-.-,other-external,65
3,202001,-30-,other-external,3690
4,202001,-30-,Art_Bell,22
...,...,...,...,...
20052453,202001,Zzzax,Brian_Banner,143
20052454,202001,Zzzax,Hulk_(film),379
20052455,202001,Zzzz,other-external,6410
20052456,202001,Zzzz,Zzz,30


In [14]:
# to verify
df_result[df_result['title']=='Gianluca_Scamacca']

,date,title,from,count
7120529,202001,Gianluca_Scamacca,other-external,3078
7120530,202001,Gianluca_Scamacca,2019_FIFA_U-20_World_Cup_squads,20
7120531,202001,Gianluca_Scamacca,2021_UEFA_European_Under-21_Championship_quali...,22
7120532,202001,Gianluca_Scamacca,2018_UEFA_European_Under-19_Championship,23
7120533,202001,Gianluca_Scamacca,U.S._Sassuolo_Calcio,81
7120534,202001,Gianluca_Scamacca,Ascoli_Calcio_1898_F.C.,87
7120535,202001,Gianluca_Scamacca,Italy_national_under-21_football_team,248
7120536,202001,Gianluca_Scamacca,2019–20_Coppa_Italia,333


In [15]:
df_result.to_csv('Results/clickstream-'+date+'.csv')

In [ ]:
# from pandas.io import sql
# import MySQLdb

# con = MySQLdb.connect()
# df_result.to_sql(con=con, name='clickstream-'+date, if_exists='replace', flavor='mysql')